# 0. Loading

First we activate the `examples` environment and then we load a couple of packages

In [ ]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

using Revise, PtyLab, Parameters, EllipsisNotation, Colors
using ImageShow, ImageCore, FFTW, BenchmarkTools, ImageIO, Downloads, HDF5, Plots, ColorSchemes
FFTW.set_num_threads(4)

# 1. Data Download

It is ~350MB large, so please be patient for a few moments (or minutes).
The file is stored temporarily somewhere else. So if you restart this notebook, it downloads it again.
However, you can also manually specify the file path down below.

In [ ]:
dataset = Downloads.download("https://figshare.com/ndownloader/files/38419391", verbose=true)

# 2. Load into ExperimentalDataCPM

In [ ]:
datah5 = h5open(dataset, "r")

In [ ]:
read(datah5, "dxd")

In [ ]:
read(datah5, "zo")# .* read(datah5, "zo") ./ (240 .* read(datah5, "dxd")) * 240

In [ ]:
(-reconstruction.Np/2:reconstruction.Np/2-1)*reconstruction.dxp

In [ ]:
reconstruction.xp

In [ ]:
experimentalData = ExperimentalDataCPM(dataset);

In [ ]:
simshow(experimentalData.ptychogram[:, :, 1], γ=0.3)

In [ ]:
simshow(experimentalData.ptychogram[:, :, 1001], γ=0.3)

In [ ]:
experimentalData.zo

# 3. Load into ReconstructionCPM

In [ ]:
reconstruction = ReconstructionCPM(experimentalData, cuda=false);

# 4. Choose engine

In [ ]:
params = Params(fftshiftFlag = false, 
                 transposePtychogram = false, 
                 comStabilizationSwitch = true)

engine = PtyLab.ePIE(betaProbe = 0.25f0, betaObject = 0.25f0, numIterations = 4000)

In [ ]:
reconstruction.npsm = 1

In [ ]:
reconstruction.entrancePupilDiameter = reconstruction.Np/3 * reconstruction.dxp

# 5. Reconstruct

In [ ]:
reconstruction.dxo

In [ ]:
reconstruction.wavelength * reconstruction.zo / reconstruction.Ld

In [ ]:
reconstruction = PtyLab.initializeObjectProbe!(reconstruction);

In [ ]:
#reconstruction.probe .+= randn(size(reconstruction.probe)) .* 0.1;

In [ ]:
@time p, o = PtyLab.reconstruct(engine, params, reconstruction);

In [ ]:
simshow(reconstruction.probe[:, :, 1,1,1,1])

In [ ]:
simshow(reconstruction.object[:, :, 1,1,1,1])